# Hyperspectral Data Mask Visualization

This notebook demonstrates how to visualize:
1. Original hyperspectral data
2. Data with mask overlay
3. Data only under the mask (masked data)

Prerequisites:
- You have already created a mask using the MaskingTool and saved it as an NPY file
- You have the original hyperspectral data file


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
import sys

# Add parent directory to path to import HyperspectralDataLoader
from HyperspectralDataLoader import HyperspectralDataLoader

SyntaxError: invalid syntax (2038356202.py, line 10)

## 1. Load Hyperspectral Data

First, load your hyperspectral data using the HyperspectralDataLoader class.


In [ ]:
# Path to your hyperspectral data
# This could be a directory containing .im3 files or a pickle file
data_path = "path/to/your/data"  # Replace with your data path

# Load the data
loader = HyperspectralDataLoader(data_path)
loader.load_data()

# Print summary of the loaded data
loader.print_summary()


## 2. Load the Mask

Load the mask that you created with the MaskingTool and saved as an NPY file.


In [ ]:
# Path to your mask NPY file
mask_path = "path/to/your/mask.npy"  # Replace with your mask file path

# Load the mask
mask = np.load(mask_path)

# Print mask information
print(f"Mask shape: {mask.shape}")
print(f"Mask type: {mask.dtype}")
print(f"Number of True values (pixels inside mask): {np.sum(mask)}")
print(f"Percentage of image covered by mask: {np.sum(mask) / mask.size * 100:.2f}%")


## 3. Visualize Original Data

Create a visualization of the original hyperspectral data without any mask applied.


In [ ]:
# Select an excitation wavelength to visualize
# You can get available excitations from loader.data_dict['metadata']['excitations']
excitation = None  # Replace with your excitation wavelength (e.g., 405.0)

# Get available excitations if not specified
if excitation is None:
    available_excitations = loader.data_dict['metadata']['excitations']
    print(f"Available excitations: {available_excitations}")
    excitation = available_excitations[0]  # Use the first available excitation
    print(f"Using excitation: {excitation} nm")

# Visualize the original data using false color
plt.figure(figsize=(10, 8))
fig = loader.visualize_false_color(excitation, method='rgb', percentile=99)
plt.title(f"Original Data (Excitation: {excitation} nm)")
plt.show()


## 4. Visualize Data with Mask Overlay

Create a visualization of the original data with the mask overlaid on top.


In [ ]:
def visualize_with_mask_overlay(loader, excitation, mask, method='rgb', percentile=99, alpha=0.5):
    """
    Visualize hyperspectral data with a mask overlay.
    
    Args:
        loader: HyperspectralDataLoader instance
        excitation: Excitation wavelength
        mask: Boolean mask array
        method: Visualization method ('rgb', 'max', 'mean', 'pca')
        percentile: Percentile for scaling
        alpha: Transparency of the mask overlay
        
    Returns:
        Matplotlib figure
    """
    # Create a figure with the original data
    fig, ax = plt.subplots(figsize=(12, 10))
    loader.visualize_false_color(excitation, method=method, percentile=percentile, ax=ax)
    
    # Create a red mask overlay with transparency
    mask_overlay = np.zeros((*mask.shape, 4))  # RGBA
    mask_overlay[mask, 0] = 1.0  # Red channel
    mask_overlay[mask, 3] = alpha  # Alpha channel
    
    # Overlay the mask
    ax.imshow(mask_overlay, alpha=alpha)
    
    # Add a title
    ax.set_title(f"Data with Mask Overlay (Excitation: {excitation} nm)")
    
    return fig

# Visualize data with mask overlay
fig = visualize_with_mask_overlay(loader, excitation, mask, method='rgb', alpha=0.3)
plt.show()


## 5. Visualize Data Only Under the Mask

Create a visualization showing only the data under the mask (masked data).


In [ ]:
def visualize_masked_data(loader, excitation, mask, method='rgb', percentile=99):
    """
    Visualize only the data under the mask.
    
    Args:
        loader: HyperspectralDataLoader instance
        excitation: Excitation wavelength
        mask: Boolean mask array
        method: Visualization method ('rgb', 'max', 'mean', 'pca')
        percentile: Percentile for scaling
        
    Returns:
        Matplotlib figure
    """
    # Get the data cube
    cube, wavelengths = loader.get_cube(excitation)
    
    # Create a masked version of the cube
    masked_cube = np.copy(cube)
    
    # Apply mask to each wavelength band
    for i in range(masked_cube.shape[2]):  # Assuming shape is (height, width, bands)
        masked_cube[:, :, i] = np.where(mask, masked_cube[:, :, i], np.nan)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Create visualization based on method
    if method == 'rgb':
        # Use three wavelengths as RGB channels
        num_bands = masked_cube.shape[2]
        if num_bands >= 3:
            indices = [int(num_bands * 0.2), int(num_bands * 0.5), int(num_bands * 0.8)]
            r_idx, g_idx, b_idx = indices
            
            r_band = masked_cube[:, :, r_idx]
            g_band = masked_cube[:, :, g_idx]
            b_band = masked_cube[:, :, b_idx]
            
            # Scale each channel
            r_scaled = r_band / np.nanpercentile(r_band, percentile)
            g_scaled = g_band / np.nanpercentile(g_band, percentile)
            b_scaled = b_band / np.nanpercentile(b_band, percentile)
            
            # Clip to [0, 1]
            r_scaled = np.clip(r_scaled, 0, 1)
            g_scaled = np.clip(g_scaled, 0, 1)
            b_scaled = np.clip(b_scaled, 0, 1)
            
            # Create RGB image
            rgb = np.stack([r_scaled, g_scaled, b_scaled], axis=2)
            
            # Create a mask for areas outside the mask
            mask_3d = np.stack([mask, mask, mask], axis=2)
            
            # Set background to black
            rgb = np.where(mask_3d, rgb, 0)
            
            ax.imshow(rgb)
            ax.set_title(f'Masked Data Only (Ex: {excitation}nm)\n'
                         f'R: {wavelengths[r_idx]}nm, G: {wavelengths[g_idx]}nm, B: {wavelengths[b_idx]}nm')
        else:
            raise ValueError("Not enough bands for RGB visualization. Try 'max' or 'mean' method.")
    
    elif method == 'max':
        # Create a color image from max projection along wavelength
        max_proj = np.nanmax(masked_cube, axis=2)  # Max along emission wavelength axis
        vmax = np.nanpercentile(max_proj, percentile)
        
        # Create a custom colormap with black background for NaN values
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('black', 1.0)
        
        ax.imshow(max_proj, cmap=cmap, vmin=0, vmax=vmax)
        ax.set_title(f'Masked Data Only - Max Projection (Ex: {excitation}nm)')
        plt.colorbar(ax.images[0], ax=ax, label='Max Intensity')
    
    elif method == 'mean':
        # Create a color image from mean projection along wavelength
        mean_proj = np.nanmean(masked_cube, axis=2)  # Mean along emission wavelength axis
        vmax = np.nanpercentile(mean_proj, percentile)
        
        # Create a custom colormap with black background for NaN values
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('black', 1.0)
        
        ax.imshow(mean_proj, cmap=cmap, vmin=0, vmax=vmax)
        ax.set_title(f'Masked Data Only - Mean Projection (Ex: {excitation}nm)')
        plt.colorbar(ax.images[0], ax=ax, label='Mean Intensity')
    
    else:
        raise ValueError(f"Method {method} not implemented for masked data visualization.")
    
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    
    plt.tight_layout()
    return fig

# Visualize only the data under the mask
fig = visualize_masked_data(loader, excitation, mask, method='rgb')
plt.show()


## 6. Display All Three Visualizations Side by Side

Create a figure with all three visualizations side by side for easy comparison.


In [ ]:
def display_all_visualizations(loader, excitation, mask, method='rgb', percentile=99):
    """
    Display all three visualizations side by side.
    
    Args:
        loader: HyperspectralDataLoader instance
        excitation: Excitation wavelength
        mask: Boolean mask array
        method: Visualization method ('rgb', 'max', 'mean')
        percentile: Percentile for scaling
    """
    # Create a figure with three subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Original data
    loader.visualize_false_color(excitation, method=method, percentile=percentile, ax=axes[0])
    axes[0].set_title(f"Original Data\n(Excitation: {excitation} nm)")
    
    # 2. Data with mask overlay
    loader.visualize_false_color(excitation, method=method, percentile=percentile, ax=axes[1])
    
    # Create a red mask overlay with transparency
    mask_overlay = np.zeros((*mask.shape, 4))  # RGBA
    mask_overlay[mask, 0] = 1.0  # Red channel
    mask_overlay[mask, 3] = 0.3  # Alpha channel
    
    # Overlay the mask
    axes[1].imshow(mask_overlay, alpha=0.3)
    axes[1].set_title(f"Data with Mask Overlay\n(Excitation: {excitation} nm)")
    
    # 3. Data only under the mask
    # Get the data cube
    cube, wavelengths = loader.get_cube(excitation)
    
    # Create a masked version of the cube
    masked_cube = np.copy(cube)
    
    # Apply mask to each wavelength band
    for i in range(masked_cube.shape[2]):  # Assuming shape is (height, width, bands)
        masked_cube[:, :, i] = np.where(mask, masked_cube[:, :, i], np.nan)
    
    # Create visualization based on method
    if method == 'rgb':
        # Use three wavelengths as RGB channels
        num_bands = masked_cube.shape[2]
        if num_bands >= 3:
            indices = [int(num_bands * 0.2), int(num_bands * 0.5), int(num_bands * 0.8)]
            r_idx, g_idx, b_idx = indices
            
            r_band = masked_cube[:, :, r_idx]
            g_band = masked_cube[:, :, g_idx]
            b_band = masked_cube[:, :, b_idx]
            
            # Scale each channel
            r_scaled = r_band / np.nanpercentile(r_band, percentile)
            g_scaled = g_band / np.nanpercentile(g_band, percentile)
            b_scaled = b_band / np.nanpercentile(b_band, percentile)
            
            # Clip to [0, 1]
            r_scaled = np.clip(r_scaled, 0, 1)
            g_scaled = np.clip(g_scaled, 0, 1)
            b_scaled = np.clip(b_scaled, 0, 1)
            
            # Create RGB image
            rgb = np.stack([r_scaled, g_scaled, b_scaled], axis=2)
            
            # Create a mask for areas outside the mask
            mask_3d = np.stack([mask, mask, mask], axis=2)
            
            # Set background to black
            rgb = np.where(mask_3d, rgb, 0)
            
            axes[2].imshow(rgb)
            axes[2].set_title(f'Data Only Under Mask\n(Excitation: {excitation} nm)')
        else:
            raise ValueError("Not enough bands for RGB visualization. Try 'max' or 'mean' method.")
    
    elif method == 'max':
        # Create a color image from max projection along wavelength
        max_proj = np.nanmax(masked_cube, axis=2)  # Max along emission wavelength axis
        vmax = np.nanpercentile(max_proj, percentile)
        
        # Create a custom colormap with black background for NaN values
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('black', 1.0)
        
        axes[2].imshow(max_proj, cmap=cmap, vmin=0, vmax=vmax)
        axes[2].set_title(f'Data Only Under Mask\n(Excitation: {excitation} nm)')
    
    elif method == 'mean':
        # Create a color image from mean projection along wavelength
        mean_proj = np.nanmean(masked_cube, axis=2)  # Mean along emission wavelength axis
        vmax = np.nanpercentile(mean_proj, percentile)
        
        # Create a custom colormap with black background for NaN values
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('black', 1.0)
        
        axes[2].imshow(mean_proj, cmap=cmap, vmin=0, vmax=vmax)
        axes[2].set_title(f'Data Only Under Mask\n(Excitation: {excitation} nm)')
    
    else:
        raise ValueError(f"Method {method} not implemented for masked data visualization.")
    
    # Set labels for all axes
    for ax in axes:
        ax.set_xlabel('Column')
        ax.set_ylabel('Row')
    
    plt.tight_layout()
    return fig

# Display all three visualizations side by side
fig = display_all_visualizations(loader, excitation, mask, method='rgb')
plt.show()


## 7. Try Different Visualization Methods

You can try different visualization methods ('rgb', 'max', 'mean') to see which one works best for your data.


In [ ]:
# Try different visualization methods
for method in ['rgb', 'max', 'mean']:
    print(f"\nVisualization method: {method}")
    fig = display_all_visualizations(loader, excitation, mask, method=method)
    plt.show()


## 8. Save Visualizations

You can save the visualizations to files for later use.


In [ ]:
def save_visualizations(loader, excitation, mask, output_dir="./visualizations", method='rgb'):
    """
    Save all three visualizations to files.
    
    Args:
        loader: HyperspectralDataLoader instance
        excitation: Excitation wavelength
        mask: Boolean mask array
        output_dir: Directory to save the visualizations
        method: Visualization method ('rgb', 'max', 'mean')
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Save original data visualization
    fig = plt.figure(figsize=(10, 8))
    loader.visualize_false_color(excitation, method=method, percentile=99)
    plt.title(f"Original Data (Excitation: {excitation} nm)")
    plt.savefig(os.path.join(output_dir, f"original_data_{method}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # 2. Save data with mask overlay
    fig = visualize_with_mask_overlay(loader, excitation, mask, method=method)
    plt.savefig(os.path.join(output_dir, f"data_with_mask_overlay_{method}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # 3. Save data only under mask
    fig = visualize_masked_data(loader, excitation, mask, method=method)
    plt.savefig(os.path.join(output_dir, f"data_under_mask_{method}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # 4. Save all visualizations side by side
    fig = display_all_visualizations(loader, excitation, mask, method=method)
    plt.savefig(os.path.join(output_dir, f"all_visualizations_{method}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved all visualizations to {output_dir}")

# Uncomment to save visualizations
# save_visualizations(loader, excitation, mask, output_dir="./visualizations", method='rgb')


## Summary

This notebook demonstrated how to:
1. Load hyperspectral data using the HyperspectralDataLoader
2. Load a mask created with the MaskingTool
3. Visualize the original data
4. Visualize the data with a mask overlay
5. Visualize only the data under the mask
6. Display all three visualizations side by side
7. Try different visualization methods
8. Save the visualizations to files

You can modify the code to suit your specific needs, such as changing the visualization method or adjusting the transparency of the mask overlay.